In [1]:
######################################################################
###############   IMPORTING THE NECESSARY PACKAGES  ##################
######################################################################

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [35]:
######################################################################
###############   DEFINING THE HYPERPARAMETERS  ######################
######################################################################

batch_size = 100
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 1
num_classes = 10
num_iter = 4
learning_rate = 0.01

In [36]:
######################################################################
##################   DEVICE CONFIGURATION ############################
######################################################################

device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [37]:
######################################################################
####################   LOADING THE DATASET  ##########################
######################################################################

# Downloading the MNIST dataset
train_dataset = torchvision.datasets.MNIST('./data/', train = True, transform = transforms.ToTensor(), download = True)
test_dataset = torchvision.datasets.MNIST('./data/', train = False, transform = transforms.ToTensor())

# DataLoading step 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)


In [38]:
######################################################################
####################  DEFINING THE MODEL  ############################
######################################################################

# Defining the modular class for the model 

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size, num_classes)

    ## Defining the forward method
    
    def forward(self,x):
        # Defining the initial hidden state and initial cell state
        c0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)
        h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)
        
        # Forward propagate LSTM
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        out, _ = self.lstm(x, (c0,h0))
        
        # Decoding the hidden state at the last time_stamp
        out = self.fc(out[:,-1,:])
        return out

# Defining the model as an instance to the class RNN
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [39]:
######################################################################
###########  DEFINING THE LOSS CRITERION AND OPTIMIZER ###############
######################################################################

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [40]:
######################################################################
####################### THE TRAINING LOOP ############################
######################################################################

total_step = len(train_loader)
for iter in range(num_iter):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward Pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)% 300 == 0:
            print("Epochs: [{}/{}], step: [{}/{}], Loss: {}".format(iter+1,num_iter,i+1, total_step, loss.item()))
    

Epochs: [1/4], step: [300/600], Loss: 0.13065467774868011
Epochs: [1/4], step: [600/600], Loss: 0.16951213777065277
Epochs: [2/4], step: [300/600], Loss: 0.018430182710289955
Epochs: [2/4], step: [600/600], Loss: 0.08410560339689255
Epochs: [3/4], step: [300/600], Loss: 0.026027603074908257
Epochs: [3/4], step: [600/600], Loss: 0.03004399500787258
Epochs: [4/4], step: [300/600], Loss: 0.017285894602537155
Epochs: [4/4], step: [600/600], Loss: 0.08444494009017944


In [41]:
######################################################################
########################### VALIDATION ###############################
######################################################################

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1,sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100*correct/total))
    

Test Accuracy of the model on the 10000 test images: 97.53 %


In [42]:
######################################################################
######################  SAVING THE MODEL #############################
######################################################################
torch.save(model.state_dict(), 'RNN_on_MNIST.ckpt')
